In [2]:
from scipy.stats import levy_stable
import numpy as np
import torch
import math
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import optim
import os

In [12]:
PATH_base = './3fcn-MNIST-unif'

In [13]:
class simpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [14]:
batch_size= 100
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

nodes = 1
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified


In [15]:
# uniform lr
lr_list = [0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12]
train_accuracy = []
train_error = []
for node in range(nodes):
    tmp_err = []
    tmp_acc = []
    print(node)
    for i in range(len(lr_list)):
        PATH = PATH_base + '/LR' + '{}'.format(i)
        # tmp_path_mod = PATH  + '/node' +'{}'.format(node)+ '/model-1' +'.pth'
        tmp_path_mod = PATH  + '/model1000' +'.pth'
        # tmp_path_mod = PATH + '/model900' +'.pth'
        tmp_net = torch.load(tmp_path_mod,map_location='cpu')
        test_acc = 0
        error = 0
        for data in train_loader:
            img, label = data
            # img=img.view(img.size(0),-1)
            img = Variable(img)
            label = Variable(label)
            out = tmp_net(img)
            loss = criterion(out, label)
            error += loss.data.item()
            _, pred = torch.max(out.data, 1)
            test_acc += pred.eq(label.view_as(pred)).sum().item()
        tmp_err.append(error/50000)
        tmp_acc.append(test_acc/50000)
        print(i)
    train_error.append(tmp_err)
    train_accuracy.append(tmp_acc)

test_accuracy = []
test_error = []
for node in range(nodes):
    tmp_acc = []
    tmp_err = []
    print(node)
    for i in range(len(lr_list)):
        PATH = PATH_base + '/LR' + '{}'.format(i)
        # tmp_path_mod = PATH  + '/node' +'{}'.format(node)+ '/model-1' +'.pth'
        tmp_path_mod = PATH  + '/model1000' +'.pth'
        # tmp_path_mod = PATH + '/model900' +'.pth'
        tmp_net = torch.load(tmp_path_mod,map_location='cpu')
        test_acc = 0
        error = 0
        for data in test_loader:
            img, label = data
            # img=img.view(img.size(0),-1)
            img = Variable(img)
            label = Variable(label)
            out = tmp_net(img)
            loss = criterion(out, label)
            error += loss.data.item()
            _, pred = torch.max(out.data, 1)
            test_acc += pred.eq(label.view_as(pred)).sum().item()
        tmp_acc.append(test_acc/10000)
        tmp_err.append(error/10000)
        print(i)
    test_error.append(tmp_err)
    test_accuracy.append(tmp_acc)

0
0
1
2
3
4
5
6
7
0
0
1
2
3
4
5
6
7
